In [1]:
"""
    Load metadata
"""

import json

files = []
labels = []
persons = ["Ashima Gupta", "Achinth Venkat", "Oishe Farhan", "Liu Edward"]
with open('../../datasets/raw/users.json') as jsonfile:
    metadata = json.load(jsonfile)
    for person in persons:
        #print(person)
        for session in metadata[person]['sessions']:
            activity = metadata[person]['sessions'][session]['activity']
            #print(activity, end=': ')
            #print(session)
            files.append(session)
            labels.append(activity)
        #print()
print(files)
print(labels)

['1552683620495_recording.csv', '1552684135826_recording.csv', '1552684518897_recording.csv', '1552684885683_recording.csv', '1552685274491_recording.csv', '1552686579599_recording.csv', '1552687068355_recording.csv', '1552687461872_recording.csv', '1552687730763_recording.csv', '1552688006554_recording.csv', '1552684886225_recording.csv', '1552685478381_recording.csv', '1552685952146_recording.csv', '1552686877018_recording.csv', '1552687454762_recording.csv', '1552687746756_recording.csv', '1552882289936_recording.csv', '1550862070278_recording.csv', '1551117337203_recording.csv', '1551117646871_recording.csv', '1551118061985_recording.csv', '1551118320943_recording.csv', '1551249571278_recording.csv', '1553039903059_recording.csv', '1553040206684_recording.csv', '1553040672641_recording.csv', '1553041018556_recording.csv', '1553041309127_recording.csv', '1553041571909_recording.csv', '1553041964221_recording.csv', '1553042270815_recording.csv', '1553042628553_recording.csv', '155268

In [2]:
"""
    Load file-by-file --> split into 'time' second intervals
"""

import numpy as np
import pandas as pd

fs = 256
trim = 5 # seconds
time = 15 # seconds
overlap = 2 #seconds

raw_path = '../../datasets/raw/'
raw_topath = '../../datasets/pulse/'

pulse_files = []
pulse_labels = []
for idx, file in enumerate(files):
    print("Reading CSV {} - {} of {}: ".format(file, idx+1, len(files)))
    df = pd.read_csv(raw_path+file)
    
    df = df.loc[:, ['time', 'TP9', 'AF7', 'AF8', 'TP10']]
    
    print("\tTruncating DataFrame: ", end='...')
    df = df.truncate(before=trim*fs, after=df.shape[0]-trim*fs-1)
    print(" Complete.")
    
    print("\tSplitting DataFrame: ", end='...')
    start = 0
    for end in range(time*fs, df.shape[0], time*fs-1):
        if end > df.shape[0]:
            break
        
        if start == 0:
            pulse_files.append(df[start:end+overlap*fs-1])
        elif end+overlap*fs > df.shape[0]:
            pulse_files.append(df[df.shape[0]-(time+overlap)*fs:df.shape[0]-1])
        else:
            pulse_files.append(df[start-overlap*fs:end])
        
        if labels[idx] == 'Simon':
            pulse_labels.append(1)
        else:
            pulse_labels.append(0)
        start = end
    print(" Complete.")
    print("Complete {} - {} of {}.".format(file, idx+1, len(files)))

print("Saving New DataFrames & Labels: ", end='...')
for idx, pulse in enumerate(pulse_files):
    name = raw_topath+str(idx)+'.csv'
    pulse.to_csv(path_or_buf=name, index=False)
pulse_labels = np.array(pulse_labels)
np.save(file='../../datasets/labels.npy', arr=pulse_labels)
print(" Complete.")

Reading CSV 1552683620495_recording.csv - 1 of 39: 
	Truncating DataFrame: ... Complete.
	Splitting DataFrame: ... Complete.
Complete 1552683620495_recording.csv - 1 of 39.
Reading CSV 1552684135826_recording.csv - 2 of 39: 
	Truncating DataFrame: ... Complete.
	Splitting DataFrame: ... Complete.
Complete 1552684135826_recording.csv - 2 of 39.
Reading CSV 1552684518897_recording.csv - 3 of 39: 
	Truncating DataFrame: ... Complete.
	Splitting DataFrame: ... Complete.
Complete 1552684518897_recording.csv - 3 of 39.
Reading CSV 1552684885683_recording.csv - 4 of 39: 
	Truncating DataFrame: ... Complete.
	Splitting DataFrame: ... Complete.
Complete 1552684885683_recording.csv - 4 of 39.
Reading CSV 1552685274491_recording.csv - 5 of 39: 
	Truncating DataFrame: ... Complete.
	Splitting DataFrame: ... Complete.
Complete 1552685274491_recording.csv - 5 of 39.
Reading CSV 1552686579599_recording.csv - 6 of 39: 
	Truncating DataFrame: ... Complete.
	Splitting DataFrame: ... Complete.
Complete 1

In [ ]:
"""
    Generate Spectrogram Combinations
"""

import numpy as np
import EEGrunt
import PIL

PSD_per_Hz_lst = []

source = 'muse-lsl'
pulse_path = '../../datasets/pulse/'
pulse_topath = '../../datasets/spectrogram/'
title = 'Muse'
file_postfix = '.csv'
bp_filter = [1, 50]

image_prefix = "./plots/Muse_Spectrogram_Channel_"
image_postfix ='.jpg'
image_lst = [image_prefix+'1'+image_postfix, image_prefix+'2'+image_postfix, image_prefix+'3'+image_postfix, image_prefix+'4'+image_postfix]

for idx in range(len(pulse_files)):
    
    # Initalization
    file = str(idx)+file_postfix
    MuseSess = EEGrunt.EEGrunt(pulse_path, file, source, title)
    MuseSess.plot = 'save'
    MuseSess.load_data()
    
    # Process
    print("Processing file {}, {} of {}: ...".format(file, idx+1, len(pulse_files)), end='')
    for channel in MuseSess.channels:
        MuseSess.load_channel(channel)
        MuseSess.notch_mains_interference()
        MuseSess.data = MuseSess.bandpass(bp_filter[0], bp_filter[1])
        MuseSess.get_spectrum_data()
        MuseSess.spectrogram()
    
    images = [ PIL.Image.open(i) for i in image_lst ]
    # pick the image which is the smallest, and resize the others to match it (can be arbitrary image shape here)
    min_shape = sorted( [(np.sum(i.size), i.size ) for i in images])[0][1]
    images_comb = np.vstack( (np.asarray( i.resize(min_shape) ) for i in images ) )
    images_comb = PIL.Image.fromarray(images_comb)
    images_comb.save(pulse_topath+str(idx)+image_postfix)
    
    # TP10 only
    PSD_per_Hz = np.mean(MuseSess.spec_PSDperHz,1)
    PSD_per_Hz = 10.0*np.log10(PSD_per_Hz[bp_filter[0]:bp_filter[1]])
    PSD_per_Hz_lst.append(PSD_per_Hz)
    print(" Complete.")
PSD_per_Hz_lst = np.array(PSD_per_Hz_lst)
np.save(file='../../datasets/PSD_per_Hz_TP10.npy', arr=PSD_per_Hz_lst)

In [3]:
"""
    Generate PSD per Hz
"""

import numpy as np
import EEGrunt

PSD_per_Hz_lst = [] # (file, channel, values)

source = 'muse-lsl'
pulse_path = '../../datasets/pulse/'
title = 'Muse'
file_postfix = '.csv'
bp_filter = [1, 50]

for idx in range(len(pulse_files)):
    
    # Initalization
    file = str(idx)+file_postfix
    MuseSess = EEGrunt.EEGrunt(pulse_path, file, source, title)
    MuseSess.plot = 'show'
    MuseSess.load_data()
    
    # Process
    print("Processing file {}, {} of {}: ...".format(file, idx+1, len(pulse_files)), end='')
    channels = []
    for channel in MuseSess.channels:
        MuseSess.load_channel(channel)
        MuseSess.notch_mains_interference()
        MuseSess.data = MuseSess.bandpass(bp_filter[0], bp_filter[1])
        MuseSess.get_spectrum_data()
        
        PSD_per_Hz = np.mean(MuseSess.spec_PSDperHz,1)
        PSD_per_Hz = 10.0*np.log10(PSD_per_Hz[bp_filter[0]:bp_filter[1]])
        channels.append(PSD_per_Hz)
        
    PSD_per_Hz_lst.append(channels)
    print(" Complete.")
PSD_per_Hz_lst = np.array(PSD_per_Hz_lst)
np.save(file='../../datasets/PSD_per_Hz.npy', arr=PSD_per_Hz_lst)

Processing file 0.csv, 1 of 889: ... Complete.
Processing file 1.csv, 2 of 889: ... Complete.
Processing file 2.csv, 3 of 889: ... Complete.
Processing file 3.csv, 4 of 889: ... Complete.
Processing file 4.csv, 5 of 889: ... Complete.
Processing file 5.csv, 6 of 889: ... Complete.
Processing file 6.csv, 7 of 889: ... Complete.
Processing file 7.csv, 8 of 889: ... Complete.
Processing file 8.csv, 9 of 889: ... Complete.
Processing file 9.csv, 10 of 889: ... Complete.
Processing file 10.csv, 11 of 889: ... Complete.
Processing file 11.csv, 12 of 889: ... Complete.
Processing file 12.csv, 13 of 889: ... Complete.
Processing file 13.csv, 14 of 889: ... Complete.
Processing file 14.csv, 15 of 889: ... Complete.
Processing file 15.csv, 16 of 889: ... Complete.
Processing file 16.csv, 17 of 889: ... Complete.
Processing file 17.csv, 18 of 889: ... Complete.
Processing file 18.csv, 19 of 889: ... Complete.
Processing file 19.csv, 20 of 889: ... Complete.
Processing file 20.csv, 21 of 889: ... 

In [9]:
"""
    Generate bandpower per unit time
"""

import numpy as np
import EEGrunt

eegbands = {
    'Delta': (0, 4),
    'Theta': (4, 8),
    'Alpha': (8, 12),
    'Beta' : (12, 30),
    'Gamma': (30, 50)
}
power_per_band_lst = [] # (file, channel, bands, values) --> i.e. (file, features, timesteps):: features is channelXbands; timesteps is values

source = 'muse-lsl'
pulse_path = '../../datasets/pulse/'
title = 'Muse'
file_postfix = '.csv'

for idx in range(len(pulse_files)):
    
    # Initalization
    file = str(idx)+file_postfix
    MuseSess = EEGrunt.EEGrunt(pulse_path, file, source, title)
    MuseSess.plot = 'show'
    MuseSess.load_data()
    
    # Process
    print("Processing file {}, {} of {}: ...".format(file, idx+1, len(pulse_files)), end='')
    
    channels = []
    for channel in MuseSess.channels:
        MuseSess.load_channel(channel)
        MuseSess.notch_mains_interference()
        MuseSess.data = MuseSess.bandpass(bp_filter[0], bp_filter[1])
        MuseSess.get_spectrum_data()
        
        PSD_per_bin = MuseSess.spec_PSDperBin
        freqs = MuseSess.spec_freqs
        
        bands = []
        for band in eegbands:
            frange = (freqs > eegbands[band][0]) & (freqs < eegbands[band][1])
            bandpower = np.sqrt(np.sum(PSD_per_bin[frange], 0))
            bands.append(bandpower)
        channels.append(bands)
    power_per_band_lst.append(channels)
    print(" Complete.")
power_per_band_lst = np.array(power_per_band_lst)
np.save(file='../../datasets/Power_per_band.npy', arr=power_per_band_lst)

Processing file 0.csv, 1 of 889: ... Complete.
Processing file 1.csv, 2 of 889: ... Complete.
Processing file 2.csv, 3 of 889: ... Complete.
Processing file 3.csv, 4 of 889: ... Complete.
Processing file 4.csv, 5 of 889: ... Complete.
Processing file 5.csv, 6 of 889: ... Complete.
Processing file 6.csv, 7 of 889: ... Complete.
Processing file 7.csv, 8 of 889: ... Complete.
Processing file 8.csv, 9 of 889: ... Complete.
Processing file 9.csv, 10 of 889: ... Complete.
Processing file 10.csv, 11 of 889: ... Complete.
Processing file 11.csv, 12 of 889: ... Complete.
Processing file 12.csv, 13 of 889: ... Complete.
Processing file 13.csv, 14 of 889: ... Complete.
Processing file 14.csv, 15 of 889: ... Complete.
Processing file 15.csv, 16 of 889: ... Complete.
Processing file 16.csv, 17 of 889: ... Complete.
Processing file 17.csv, 18 of 889: ... Complete.
Processing file 18.csv, 19 of 889: ... Complete.
Processing file 19.csv, 20 of 889: ... Complete.
Processing file 20.csv, 21 of 889: ... 